# SQLite for Data Scientists

#### Produced & Presented by Florents Tselai - [tselai.com](tselai.com)

## 5. ETL with Triggers

An (SQLite) trigger is a named database object that is executed automatically when an INSERT, UPDATE or DELETE statement is issued against the associated table.

* Triggers can be used to specify ETL routines without paying the IO cost.
* They are a simple way to leverage the so-called *pushdown optimization* (send the logic to the data)
* They also ensure ACID-compliance

They are generally regarded "obscure" for non-DBAs but can be great tool for a data practitioner's toolbelt

In [1]:
import json
import pandas as pd
from gzip import GzipFile

In [2]:
with GzipFile('../data/hn_dump.json.gz', 'r') as fin:
    data = json.loads(fin.read().decode('utf-8'))

In [3]:
data[0]

{'created_at': '2012-10-05T13:51:25.000Z',
 'title': '',
 'url': '',
 'author': 'chris_wot',
 'points': 136,
 'story_text': None,
 'comment_text': 'One of my proudest moments was finding a bug in SQLite where a corrupted index caused a select statement to segfault Firefox.<p>I jumped through a <i>lot</i> of hoops to get to the point where I got a backtrace that showed me the SQL statement of a corrupted places.sqlite. I then loaded SQLite on the data file, ran the statement and reproduced the segfault. One of their lead devs then got in contact with me, grabbed the data file and fixed the issue.<p>I suspect that not only did my diagnosis lead to a fix for a LOT of Firefox crashes, but it stopped a lot of frustrating crashes on things like iPhones, etc :-)<p>I may not have done the fix, but I took the time to reproduce the problem. It felt damn good :-)<p>P.S. in case anyone is interested, the bug is <a href="https://bugzilla.mozilla.org/show_bug.cgi?id=581946" rel="nofollow">https://bu

Let's create a *table* to store "summarized" view of the data

In [4]:
from sqlite3 import connect
DB_PATH = '../sqlite-olt.db'

with connect(DB_PATH) as db:
        db.execute("""
        create table if not exists items (
            title      text,
            points     int,
            item_id    text primary key,
            item_url   text,
            created_at timestamp
        );
        """)

Triggers usually follow a quite fine-grained syntax

```
CREATE TRIGGER [IF NOT EXISTS] trigger_name 
   [BEFORE|AFTER|INSTEAD OF] [INSERT|UPDATE|DELETE] 
   ON table_name
   [WHEN condition]
BEGIN
 statements;
END;
```

We want to just dump json data on the `hn_raw_data` table and have the summarized view (i.e. selected fields) pushed to the `items` table.

In [5]:
with connect(DB_PATH) as db:
    db.execute("""
            drop trigger if exists clean_hn_items_raw;
    """)
    db.execute("""
        create trigger if not exists clean_hn_items_raw
            after
        insert
        on hn_items_raw
            for each row
        begin
        insert into items
        values (json_extract(new.data, '$.title'), json_extract(new.data, '$.points'), json_extract(new.data, '$.objectID'),
                "https://news.ycombinator.com/item?id=" || json_extract(new.data, '$.objectID'),
                json_extract(new.data, '$.created_at'))
        on conflict do nothing;
        end
    """)

In [6]:
import json
with connect(DB_PATH) as db:
    db.execute("insert into hn_items_raw(data) values (?)", (json.dumps(data[0]),))

What about performance ?

### Question: 

How could we use a trigger to automatically have indexed comments for FTS as we did in the previous section ?

In [7]:
with connect(DB_PATH) as db:
    db.execute("""
            drop trigger if exists do_index_comments_text;
    """)
    db.execute("""
        create trigger if not exists do_index_comments_text
            after
        insert
        on hn_items_raw
            for each row
        begin
        insert into comments_fts
        values (json_extract(new.data, '$.objectID'), 
        json_extract(new.data, '$.author'), json_extract(new.data, '$.comment_text'));
        end
    """)

In [8]:
with connect(DB_PATH) as db:
    db.execute("delete from comments_fts")  

In [9]:
with connect(DB_PATH) as db:
    for item in data[:1000]:
        db.execute("insert into hn_items_raw(data) values (?)", (json.dumps(item),))